In [1]:
from bs4 import BeautifulSoup as bs
import re
import requests
from multiprocessing import Pool, Queue

In [2]:
data = []
with open("./brassicaceae.tsv") as f:
    while True:
        line = f.readline()
        try: 
            data.append([line.split("\t")[0].strip(), line.split("\t")[1].strip()])
        except IndexError:
            data.append([line.split("\t")[0].strip(), "-"])
        if not line: break

In [3]:
woodiness = r"(.*nnual.*)" # |(.*erennial.*)"
def crawl_once(element):
    try: 
        req = requests.get(f"http://worldfloraonline.org/taxon/{element[0]}", timeout=30)
    except requests.exceptions.ConnectTimeout:
        return None
    soup = bs(req.text, "html.parser")
    general = soup.select("#general")
    morphology = soup.select("#morphology")
    if (bool(re.compile(woodiness).search(str(general))) or bool(re.compile(woodiness).search(str(morphology)))): 
        return element[1]

In [4]:
pool = Pool(processes=80)
woody_naive = pool.map(crawl_once, data)
woody = []
for e in woody_naive:
    if e != None: woody.append(e)
# del woody_naive

In [5]:
with open("AnnualBrassicaceae.txt", "w") as f:
    for w in woody:
        f.write(f"{w}\n")